# Keras MobileNet Benchmark
(Note: Forked from Beluga : [https://www.kaggle.com/gaborfodor/greyscale-mobilenet-lb-0-892])

In a previous benchmark we used a simple three layer ConvNet. This time we use a deeper MobileNet architecture on greyscale strokes. 

This kernel has three main components:

* MobileNet
* Fast and memory efficient Image Generator with temporal 
* Full training & submission with Kaggle Kernel

I did some paramer search but it should not be hard to improve the current score.

## Setup
Import the necessary libraries and a few helper functions.

In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
import cv2
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation

from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam

from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

start = dt.datetime.now()

Using TensorFlow backend.


In [2]:
DP_DIR = '../input/shuffle-csvs/'
INPUT_DIR = '../input/'

BASE_SIZE = 256
NCSVS = 100
NCATS = 340
np.random.seed(seed=1987)
tf.set_random_seed(seed=1987)
def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)

In [3]:
def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

## MobileNet

MobileNets are based on a streamlined architecture that uses depthwise separable convolutions to build light weight deep neural networks.

[MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/pdf/1704.04861.pdf)

In [4]:
STEPS = 2000
EPOCHS = 10
BASE_SIZE = size = 128
batchsize = 256
stroke_size = 3

In [ ]:
from collections import defaultdict

NCOUNTRY = 21
def one_hot(x, classes=21):
    tmp = np.zeros(classes)
    tmp[x] = 1
    return tmp

countrycodes = ['US', 'GB', 'CA', 'DE', 'AU', 'RU', 'BR', 'SE', 'FI', 'CZ', 'IT', 'PL', 'FR', 'KR', 'TH', 'PH', 'SA', 'HU', 'NL', 'ID']
countrymap = defaultdict(lambda: NCOUNTRY-1)
for i, country in enumerate(countrycodes): countrymap[country]=i
    
def countrymapper(x):
    mapped = np.zeros((x.shape[0], NCOUNTRY))
    print(mapped.shape)
    for i, country in enumerate(x):
        mapped[i][countrymap[country]] = 1
    return mapped

In [5]:
def draw_cv2(raw_strokes, lw=stroke_size, time_color=True):
    img = np.zeros((size, size), np.uint8)
    for t in range(len(raw_strokes)-1, -1, -1):
        stroke = raw_strokes[t]
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    return img
    
def sample(strokes, downsize_to=size-2):
    division = 256 / downsize_to
    for i in range(len(strokes)):
        for j in range(len(strokes[i])):
            for k in range(len(strokes[i][j])):
                strokes[i][j][k] = round(strokes[i][j][k]/division)+1
    return strokes

def image_generator_xd(size, batchsize, ks, lw=stroke_size, time_color=True):
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize):
                df = df[df.recognized]
                df['drawing'] = df.drawing.apply(ast.literal_eval)
                df['scaled_drawing'] = df.drawing.apply(sample)
                x = np.zeros((len(df), size, size, 1))
                for i, raw_strokes in enumerate(df.scaled_drawing.values):
                    x[i, :, :, 0] = draw_cv2(raw_strokes, lw=lw, time_color=True)
                x = preprocess_input(x).astype(np.float32)
                
#                 x2 = countrymapper(df.countrycode)
                
                y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                yield x, y
#                 yield [x, x2], y

def df_to_image_array_xd(df, size, lw=stroke_size, time_color=True):
    df['drawing'] = df.drawing.apply(ast.literal_eval)
    df['scaled_drawing'] = df.drawing.apply(sample)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.scaled_drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, lw=lw, time_color=True)
    x = preprocess_input(x).astype(np.float32)
    return x

In [6]:
valid_df = pd.read_csv(os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(NCSVS - 1)), nrows=60000)
x_valid = df_to_image_array_xd(valid_df, size)
y_valid = keras.utils.to_categorical(valid_df.y, num_classes=NCATS)
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(60000, 128, 128, 1) (60000, 340)
Validation array memory 3.66 GB


In [7]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, ks=range(NCSVS - 1))

In [ ]:
x, y = next(train_datagen)
n = 8
fig, axs = plt.subplots(nrows=n, ncols=n, sharex=True, sharey=True, figsize=(12, 12))
for i in range(n**2):
    ax = axs[i // n, i % n]
    (-x[i]+1)/2
    ax.imshow((-x[i, :, :, 0] + 1)/2, cmap=plt.cm.gray)
    ax.axis('off')
plt.tight_layout()
fig.savefig('gs.png', dpi=300)
plt.show();

In [14]:
# MobileNet(input_shape=None, alpha=1.0, depth_multiplier=1,
#           dropout=1e-3, include_top=True, weights='imagenet',
#           input_tensor=None, pooling=None, classes=1000)

base_model = MobileNet(input_shape=(size, size, 1), alpha=1.,
                  weights=None, dropout=0.2, classes=NCATS)
# base_model.load_weights("mobilenet40epochs.h5")
# base_model.trainable = False
# base_model = MobileNet(input_shape=(size, size,3), alpha=1., weights='imagenet', include_top = False)
# base_model.load_weights("mobilenet40epochs.h5")

base_model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
base_model.load_weights("MobileNet.01-1.72.hdf5")
print(base_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
__________

In [16]:
callbacks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    ModelCheckpoint("MobileNet.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_categorical_accuracy',
                    save_weights_only=True, mode='auto', period=1)
]

hists = []
hist = base_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/10
2000/2000 [==============================] - 655s 328ms/step - loss: 1.0999 - categorical_crossentropy: 1.0999 - categorical_accuracy: 0.6893 - top_3_accuracy: 0.8783 - val_loss: 1.4191 - val_categorical_crossentropy: 1.4191 - val_categorical_accuracy: 0.6619 - val_top_3_accuracy: 0.8393
Epoch 2/10
2000/2000 [==============================] - 619s 309ms/step - loss: 0.9468 - categorical_crossentropy: 0.9468 - categorical_accuracy: 0.7277 - top_3_accuracy: 0.9024 - val_loss: 1.3660 - val_categorical_crossentropy: 1.3660 - val_categorical_accuracy: 0.6740 - val_top_3_accuracy: 0.8501
Epoch 3/10
2000/2000 [==============================] - 625s 312ms/step - loss: 0.8667 - categorical_crossentropy: 0.8667 - categorical_accuracy: 0.7480 - top_3_accuracy: 0.9153 - val_loss: 1.2454 - val_categorical_crossentropy: 1.2454 - val_categorical_accuracy: 0.6966 - val_top_3_accuracy: 0.8658
Epoch 4/10
2000/2000 [==============================] - 623s 311ms/step - loss: 0.8256 - categorical

In [17]:
def print_history(hists):
    tmp = hists[-1].history
    print("\t".join(["%.3f"%tmp[key][-1]
        for key in ('loss', 'val_loss', 'val_categorical_accuracy', 'val_top_3_accuracy')]))
    print()

In [18]:
hist = base_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)
print_history(hists)

Epoch 1/10
2000/2000 [==============================] - 622s 311ms/step - loss: 0.6936 - categorical_crossentropy: 0.6936 - categorical_accuracy: 0.7930 - top_3_accuracy: 0.9386 - val_loss: 1.0461 - val_categorical_crossentropy: 1.0461 - val_categorical_accuracy: 0.7480 - val_top_3_accuracy: 0.8966
Epoch 2/10
2000/2000 [==============================] - 628s 314ms/step - loss: 0.6866 - categorical_crossentropy: 0.6866 - categorical_accuracy: 0.7949 - top_3_accuracy: 0.9396 - val_loss: 1.0232 - val_categorical_crossentropy: 1.0232 - val_categorical_accuracy: 0.7593 - val_top_3_accuracy: 0.8999
Epoch 3/10
2000/2000 [==============================] - 626s 313ms/step - loss: 0.6740 - categorical_crossentropy: 0.6740 - categorical_accuracy: 0.7981 - top_3_accuracy: 0.9415 - val_loss: 1.0174 - val_categorical_crossentropy: 1.0174 - val_categorical_accuracy: 0.7568 - val_top_3_accuracy: 0.9006
Epoch 4/10
2000/2000 [==============================] - 623s 312ms/step - loss: 0.6666 - categorical

In [19]:
hist = base_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)
print_history(hists)

Epoch 1/10
2000/2000 [==============================] - 628s 314ms/step - loss: 0.6159 - categorical_crossentropy: 0.6159 - categorical_accuracy: 0.8134 - top_3_accuracy: 0.9489 - val_loss: 0.9718 - val_categorical_crossentropy: 0.9718 - val_categorical_accuracy: 0.7718 - val_top_3_accuracy: 0.9085
Epoch 2/10
2000/2000 [==============================] - 608s 304ms/step - loss: 0.6089 - categorical_crossentropy: 0.6089 - categorical_accuracy: 0.8149 - top_3_accuracy: 0.9498 - val_loss: 0.9652 - val_categorical_crossentropy: 0.9652 - val_categorical_accuracy: 0.7727 - val_top_3_accuracy: 0.9097
Epoch 3/10
2000/2000 [==============================] - 607s 304ms/step - loss: 0.6097 - categorical_crossentropy: 0.6097 - categorical_accuracy: 0.8155 - top_3_accuracy: 0.9495 - val_loss: 0.9635 - val_categorical_crossentropy: 0.9635 - val_categorical_accuracy: 0.7719 - val_top_3_accuracy: 0.9079
Epoch 4/10
2000/2000 [==============================] - 611s 306ms/step - loss: 0.6057 - categorical

In [20]:
hist = base_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)
print_history(hists)

Epoch 1/10
2000/2000 [==============================] - 610s 305ms/step - loss: 0.5565 - categorical_crossentropy: 0.5565 - categorical_accuracy: 0.8305 - top_3_accuracy: 0.9559 - val_loss: 0.9054 - val_categorical_crossentropy: 0.9054 - val_categorical_accuracy: 0.7880 - val_top_3_accuracy: 0.9158
Epoch 2/10
2000/2000 [==============================] - 610s 305ms/step - loss: 0.5573 - categorical_crossentropy: 0.5573 - categorical_accuracy: 0.8297 - top_3_accuracy: 0.9555 - val_loss: 0.8955 - val_categorical_crossentropy: 0.8955 - val_categorical_accuracy: 0.7911 - val_top_3_accuracy: 0.9167
Epoch 3/10
2000/2000 [==============================] - 608s 304ms/step - loss: 0.5522 - categorical_crossentropy: 0.5522 - categorical_accuracy: 0.8308 - top_3_accuracy: 0.9567 - val_loss: 0.8964 - val_categorical_crossentropy: 0.8964 - val_categorical_accuracy: 0.7903 - val_top_3_accuracy: 0.9166
Epoch 4/10
2000/2000 [==============================] - 618s 309ms/step - loss: 0.5446 - categorical

In [27]:
hist = base_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)
print_history(hists)

Epoch 1/10
2000/2000 [==============================] - 946s 473ms/step - loss: 0.4921 - categorical_crossentropy: 0.4921 - categorical_accuracy: 0.8474 - top_3_accuracy: 0.9634 - val_loss: 0.8785 - val_categorical_crossentropy: 0.8785 - val_categorical_accuracy: 0.7985 - val_top_3_accuracy: 0.9216
Epoch 2/10
2000/2000 [==============================] - 940s 470ms/step - loss: 0.4894 - categorical_crossentropy: 0.4894 - categorical_accuracy: 0.8480 - top_3_accuracy: 0.9637 - val_loss: 0.8726 - val_categorical_crossentropy: 0.8726 - val_categorical_accuracy: 0.7970 - val_top_3_accuracy: 0.9215
Epoch 3/10
2000/2000 [==============================] - 937s 468ms/step - loss: 0.4902 - categorical_crossentropy: 0.4902 - categorical_accuracy: 0.8477 - top_3_accuracy: 0.9635 - val_loss: 0.8715 - val_categorical_crossentropy: 0.8715 - val_categorical_accuracy: 0.7972 - val_top_3_accuracy: 0.9213
Epoch 4/10
2000/2000 [==============================] - 940s 470ms/step - loss: 0.4977 - categorical

In [ ]:
hist_df = pd.concat([pd.DataFrame(hist.history) for hist in hists], sort=True)
hist_df.index = np.arange(1, len(hist_df)+1)
fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
axs[0].plot(hist_df.val_categorical_accuracy, lw=5, label='Validation Accuracy')
axs[0].plot(hist_df.categorical_accuracy, lw=5, label='Training Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].grid()
axs[0].legend(loc=0)
axs[1].plot(hist_df.val_categorical_crossentropy, lw=5, label='Validation MLogLoss')
axs[1].plot(hist_df.categorical_crossentropy, lw=5, label='Training MLogLoss')
axs[1].set_ylabel('MLogLoss')
axs[1].set_xlabel('Epoch')
axs[1].grid()
axs[1].legend(loc=0)
fig.savefig('hist.png', dpi=300)
plt.show();

In [41]:
base_model.load_weights(filepath="MobileNet.10-0.86.hdf5")

In [50]:
valid_predictions = base_model.predict(x_valid, batch_size=128, verbose=1)
map3 = mapk(valid_df[['y']].values, preds2catids(valid_predictions).values)
print('Map3: {:.3f}'.format(map3))

60000/60000 [==============================] - 17s 288us/step
Map3: 0.854


In [48]:
preds2catids(valid_predictions).values

array([[143, 197, 152],
       [ 28, 186, 315],
       [298, 112,  98],
       ...,
       [ 15,  18, 269],
       [ 40, 311, 293],
       [105,  13,  26]], dtype=int64)

## Create Submission

In [30]:
test = pd.read_csv(os.path.join(INPUT_DIR, 'test_simplified.csv'))
test.head()
x_test = df_to_image_array_xd(test, size)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

,key_id,countrycode,drawing
0,9000003627287624,DE,"[[[17, 18, 20, 25, 137, 174, 242, 249, 251, 25..."
1,9000010688666847,UA,"[[[174, 145, 106, 38, 11, 4, 4, 15, 29, 78, 16..."
2,9000023642890129,BG,"[[[0, 12, 14, 17, 16, 24, 55, 57, 60, 79, 82, ..."
3,9000038588854897,US,"[[[0, 9, 23, 40, 54, 60, 81, 105, 123, 167, 20..."
4,9000052667981386,AR,"[[[87, 82, 71, 63, 66, 92, 96, 95], [220, 218,..."


(112199, 4) (112199, 128, 128, 1)
Test array memory 6.85 GB


In [31]:
test_predictions = base_model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

cats = list_all_categories()
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

112199/112199 [==============================] - 34s 306us/step


,a,b,c
0,234,281,266
1,144,36,226
2,305,62,53
3,187,303,111
4,56,113,84


(112199, 3)

,a,b,c
0,radio,stereo,snorkel
1,hockey_puck,bottlecap,pool
2,The_Great_Wall_of_China,castle,camel
3,mountain,tent,finger
4,campfire,fireplace,crown


(112199, 3)

In [32]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.to_csv('gs_mn_submission_{}.csv'.format(int(map3 * 10**4)), index=False)
submission.head()
submission.shape

,key_id,word
0,9000003627287624,radio stereo snorkel
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle camel
3,9000038588854897,mountain tent finger
4,9000052667981386,campfire fireplace crown


(112199, 2)

In [33]:
end = dt.datetime.now()
print('Latest run {}.\nTotal time {}s'.format(end, (end - start).seconds))

Latest run 2018-11-30 10:47:52.475824.
Total time 38241s
